In [25]:
import requests
from bs4 import BeautifulSoup

def increment_month(ym):
    """
    YYYYMM形式の整数を受け取り、1ヶ月進めた値を返す。
    例：202012 -> 202101, 202101 -> 202102
    """
    year = ym // 100
    month = ym % 100
    month += 1
    if month > 12:
        month = 1
        year += 1
    return year * 100 + month

def scrape_event_detail(detail_url):
    response = requests.get(detail_url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')
    
    event_data = {}
    
    # タイトル
    title_tag = soup.select_one('.eventDetail .eventTit')
    event_data['detail_title'] = title_tag.get_text(strip=True) if title_tag else None
    
    # カテゴリ
    category_tag = soup.select_one('.eventDetail .category')
    event_data['detail_category'] = category_tag.get_text(strip=True) if category_tag else None
    
    # 説明文
    disc_tag = soup.select_one('.eventDetail .disc')
    event_data['description'] = disc_tag.get_text(strip=True) if disc_tag else None
    
    # 開催期間
    date_dd = soup.select_one('.eventDetail dl.date dd')
    if date_dd:
        date_main = date_dd.select_one('.main')
        event_data['date_period'] = date_main.get_text(strip=True) if date_main else None

    # 開催時間
    time_dd = soup.select_one('.eventDetail dl.time dd')
    if time_dd:
        time_main = time_dd.select_one('.main')
        time_note = time_dd.select_one('.note')
        event_data['time_main'] = time_main.get_text(strip=True) if time_main else None
        event_data['time_note'] = time_note.get_text(strip=True) if time_note else None

    # 対象者/入場料
    person_dd = soup.select_one('.eventDetail dl.person dd .authorP')
    if person_dd:
        target_div = person_dd.select_one('.target')
        price_div = person_dd.select_one('.price')
        event_data['detail_target'] = target_div.get_text(strip=True) if target_div else None
        event_data['detail_price'] = price_div.get_text(strip=True) if price_div else None

    # 利用施設
    facility_dd = soup.select('.eventDetail dl.facility dd .hall')
    if facility_dd:
        facilities = []
        for hall in facility_dd:
            hall_name = hall.select_one('.hallName')
            hall_no = hall.select_one('.hallNo')
            facility_str = ''
            if hall_name:
                facility_str += hall_name.get_text(strip=True)
            if hall_no:
                facility_str += ' ' + hall_no.get_text(strip=True)
            facilities.append(facility_str.strip())
        event_data['facilities'] = facilities

    # 主催者
    organizer_dd = soup.select_one('.eventDetail dl.organizer dd')
    event_data['organizer'] = organizer_dd.get_text(strip=True) if organizer_dd else None

    # URL
    url_dd = soup.select_one('.eventDetail dl.contact dd .url a')
    event_data['official_url'] = url_dd.get('href') if url_dd else None

    # 詳細ページ画像
    thumb_img = soup.select_one('.eventDetail .thumb img')
    if thumb_img:
        img_src = thumb_img.get('src')
        if img_src and img_src.startswith('/'):
            img_src = 'https://www.m-messe.co.jp' + img_src
        event_data['detail_image_url'] = img_src
    else:
        event_data['detail_image_url'] = None
    
    return event_data


def scrape_events_for_month(month, pmonth, ss, max_pages=100):
    """
    指定のmonth, pmonth, ssでイベント一覧と詳細情報を取得。
    max_pagesは1ヶ月あたりに取得を試みるページ数の上限。
    イベントが取得できなくなったらbreak。
    """
    all_events = []
    for page in range(1, max_pages+1):
        url = "https://www.m-messe.co.jp/event/"
        params = {
            "month": str(month),
            "c": "",
            "word": "",
            "page": str(page),
            "pmonth": str(pmonth),
            "ss": str(ss),
        }
        response = requests.get(url, params=params)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, "html.parser")

        event_list = soup.select("ul.eventList > li.eventInr.clear")
        if not event_list:
            # イベントがなければbreak
            break

        for li in event_list:
            a_tag = li.find("a")
            if a_tag:
                detail_url = a_tag.get("href")
                if detail_url and detail_url.startswith("/event/detail/"):
                    detail_url = "https://www.m-messe.co.jp" + detail_url
                else:
                    detail_url = None
            else:
                detail_url = None

            category_div = li.select_one(".category")
            category = category_div.get_text(strip=True) if category_div else None

            date_div = li.select_one(".date")
            date_text = date_div.get_text(strip=True) if date_div else None

            title_div = li.select_one(".eventTit")
            title = title_div.get_text(strip=True) if title_div else None

            target_div = li.select_one(".target")
            target_text = target_div.get_text(strip=True).replace("対象", "") if target_div else None

            thumb_div = li.select_one(".thumb img")
            image_url = thumb_div.get("src") if thumb_div else None
            if image_url and image_url.startswith("/"):
                image_url = "https://www.m-messe.co.jp" + image_url

            event = {
                "title": title,
                "date": date_text,
                "category": category,
                "target": target_text,
                "detail_url": detail_url,
                "image_url": image_url,
                "month_param": month
            }
            
            # detail_urlがあれば詳細情報取得
            if detail_url:
                detail_data = scrape_event_detail(detail_url)
                event.update(detail_data)
            
            all_events.append(event)

    return all_events

def scrape_all_events(start_month=202012, end_month=202606):
    """
    start_monthからend_monthまで月を進めながらイベントを取得。
    ssはstart_monthから進めるたびに+1していく。
    pmonthは固定値としてend_monthを渡します。
    """
    all_events = []
    ss = 0
    pmonth = end_month

    current_month = start_month
    while current_month <= end_month:
        events = scrape_events_for_month(current_month, pmonth, ss)
        all_events.extend(events)

        # 次の月へ
        current_month = increment_month(current_month)
        ss += 1

    return all_events


# 例: 2020年12月から2021年6月までのイベントを取得
data = scrape_all_events(start_month=202012, end_month=202606)


In [27]:
data[-1]

{'title': '通信制高校・サポート校合同相談会',
 'date': '2025.11.24(月)',
 'category': 'イベント',
 'target': '一般',
 'detail_url': 'https://www.m-messe.co.jp/event/detail/7921',
 'image_url': 'https://www.m-messe.co.jp/saved/images/event/8e/63/c5f31288126de5d09a023f5150e8ebe514bf8e63.jpg',
 'month_param': 202511,
 'detail_title': '通信制高校・サポート校合同相談会',
 'detail_category': 'イベント',
 'description': '通信制高校、サポート校、高等専修学校、技能連携校等の多様な進路についての合同相談会です。各校ブースで話を聴けるほか、通信制のしくみに関する講演や在校生・卒業生の体験談、各種相談コーナーもあります。',
 'date_period': '2025年11月24日(月・祝)',
 'time_main': '10:30～15:30',
 'time_note': None,
 'facilities': ['国際会議場'],
 'organizer': '学びリンク株式会社',
 'official_url': 'https://www.stepup-school.net/soudankai/',
 'detail_image_url': 'https://www.m-messe.co.jp/cache/images/event/8e/63/c5f31288126de5d09a023f5150e8ebe514bf8e63.1200x1200.none.jpg'}

In [30]:
import pandas as pd

# 辞書型データをDataFrameに変換
df = pd.DataFrame(data)



In [34]:
df[df['month_param']==202411]

,title,date,category,target,detail_url,image_url,month_param,detail_title,detail_category,description,date_period,time_main,time_note,detail_target,detail_price,facilities,organizer,official_url,detail_image_url
995,KAMITSUBAKI WARS 2024 神椿幕張戦線 IN 幕張メッセイベントホール ...,2024.11.02(土),音楽イベント,一般,https://www.m-messe.co.jp/event/detail/7869,https://www.m-messe.co.jp/saved/images/event/4...,202411,KAMITSUBAKI WARS 2024 神椿幕張戦線 IN 幕張メッセイベントホール ...,音楽イベント,KAMITSUBAKI STUDIOによるリアル会場でのライブシリーズ「KAMITSUBAK...,2024年11月2日(土),,16:00 開場 / 17:00 開演,NaN,NaN,[幕張イベントホール],株式会社ディスクガレージ,None,https://www.m-messe.co.jp/cache/images/event/4...
996,SORARU Birthday 2024 VOCALOID Cover Live -空祭り-,2024.11.02(土) 〜 2024.11.03(日),音楽イベント,一般,https://www.m-messe.co.jp/event/detail/7887,https://www.m-messe.co.jp/saved/images/event/7...,202411,SORARU Birthday 2024 VOCALOID Cover Live -空祭り-,音楽イベント,そらる、自身の誕生日を含む幕張メッセ2daysのワンマンライブ『SORARU Birthda...,2024年11月2日(土)～2024年11月3日(日・祝),14:00～19:00,11/2(土) 16:00～19:00 (予定)11/3(日) 14:00～17:00 (予定),NaN,NaN,[国際展示場 展示ホール9〜11],cielkocka / PONY CANYON,https://info.diskgarage.com/,https://www.m-messe.co.jp/cache/images/event/7...
997,家具メッセバザールin幕張メッセ,2024.11.02(土) 〜 2024.11.04(月),イベント,一般,https://www.m-messe.co.jp/event/detail/7915,https://www.m-messe.co.jp/saved/images/event/5...,202411,家具メッセバザールin幕張メッセ,イベント,かねたや家具店が主催する家具のセール販売会です。イベント総開催数は300回を超えており、国内...,2024年11月2日(土)～2024年11月4日(月・祝),10:00～19:00,11/2(土)10:00～19:0011/3(日)10:00～19:0011/4(月)10:...,NaN,NaN,[国際展示場 展示ホール3],株式会社かねたや家具店,https://www.kanetaya.com/makuhari_messe/?afe=I...,https://www.m-messe.co.jp/cache/images/event/5...
998,第34回日本医療薬学会年会,2024.11.02(土) 〜 2024.11.04(月),イベント,関係者,https://www.m-messe.co.jp/event/detail/7885,https://www.m-messe.co.jp/static/images/event/...,202411,第34回日本医療薬学会年会,イベント,学会テーマ「未来の医療をデザインする薬学・薬剤師の視点」,2024年11月2日(土)～2024年11月4日(月・祝),,11月2日9:00～18:4011月3日9:00～18:4511月4日9:00～15:35,関係者,"会員15,000円非会員23,000円","[国際展示場 展示ホール8, 国際会議場 コンベンションホール,国際会議室,中会議室201,...",一般社団法人日本医療薬学会,https://convention.jtbcom.co.jp/34jsphcs/,None
999,冬スポ‼WINTER SPORTS FESTA season24,2024.11.02(土) 〜 2024.11.04(月),即売会,一般,https://www.m-messe.co.jp/event/detail/7881,https://www.m-messe.co.jp/saved/images/event/2...,202411,冬スポ‼WINTER SPORTS FESTA season24,即売会,ウィンタースポーツ用品の販売イベント,2024年11月2日(土)～2024年11月4日(月・祝),,11/2(土) 13:00～19:00(レジクローズ 19:30)11/3(日) 11:00...,NaN,NaN,[国際展示場 展示ホール7],株式会社ディー.オー.ディー.,https://www.fuyusupo.jp/,https://www.m-messe.co.jp/cache/images/event/2...
1000,KAMITSUBAKI WARS 2024 神椿幕張戦線 IN 幕張メッセイベントホール ...,2024.11.03(日),音楽イベント,一般,https://www.m-messe.co.jp/event/detail/7870,https://www.m-messe.co.jp/saved/images/event/7...,202411,KAMITSUBAKI WARS 2024 神椿幕張戦線 IN 幕張メッセイベントホール ...,音楽イベント,KAMITSUBAKI STUDIOによるリアル会場でのライブシリーズ「KAMITSUBAK...,2024年11月3日(日・祝),,16:00 開場 / 17:00 開演,NaN,NaN,[幕張イベントホール],株式会社ディスクガレージ,None,https://www.m-messe.co.jp/cache/images/event/7...
1001,地域力の強化に向けた全国市町村長サミット2024in千葉,2024.11.05(火),学会・会議,関係者,https://www.m-messe.co.jp/event/detail/7780,https://www.m-messe.co.jp/static/images/event/...,202411,地域力の強化に向けた全国市町村長サミット2024in千葉,学会・会議,全国の市町村長等が一堂に会して議論・交流を行うことを通じて、地域活性化の一層の進展・普及を図...,2024年11月5日(火),13:00～18:50,なし,関係者,無料,"[国際会議場 コンベンションホール,国際会議室,中会議室201,213,214,215,21...",総務省 及び 千葉県,https://www.pref.chiba.lg.jp/,None
1002,ワキタジャンボフェア2024in幕張 Build the future～共に創る未来～,2024.11.07(木) 〜 2024.11.08(金),展示会・見本市,関係者,https://www.m-messe.co.jp/event/detail/7888,https://www.m-messe.co.jp/saved/images/event/c...,202411,ワキタジャンボフェア2024in幕張 Build the future～共に創る未来～,展示会・見本市,最先端技術を目の前で実演!話題の新技術や新製品を体感してください!,2024年11月7日(木)～2024年11月8日(金),10:00～17:00,11/7(木)10:00～17:0011/8(金)9:00～16:00,関係者,ご招待での入場となります。,[国際展示場 展示ホール1],株式会社ワキタ,None,https://www.m-messe.co.jp/cache/images/event/c...
1003,Cybozu Days 2024,2024.11.07(木) 〜 2024.11.08(金),イベント,一般,https://www.m-messe.co.jp/event/detail/7872,https://www.m-messe.co.jp/saved/images/event/b...,202411,Cybozu Days 2024,イベント,サイボウズのクラウドサービス「キントーン」の事例セッションや120以上の展示ブースが集まる総...,2024年11月7日(木)～2024年11月8日(金),11:00～18:00,受付開始 10:00,NaN,NaN,[国際展示場 展示ホール4〜6],サイボウズ株式会社,h

In [ ]:
# DataFrameをCSVファイルとして保存
df.to_csv("events.csv", index=False, encoding="utf-8")

In [ ]:
#CSVデータをevent_detailsTBLへ変換する